In [54]:
using GLM
using TimeSeries, DataFrames
using XLSX
using StatsBase
using PrettyTables


In [116]:
data = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Returns";infer_eltypes=true))
data = TimeArray(data,timestamp=:Date)



615×8 TimeArray{Any, 2, Date, Matrix{Any}} 1973-01-31 to 2024-03-31
┌────────────┬────────────┬─────────────┬──────────────┬──────────────┬─────────
│            │ ACWI_Net   │ USDPLN      │ ACWI_Net_PLN │ CPI_USA      │ CPI_PL ⋯
├────────────┼────────────┼─────────────┼──────────────┼──────────────┼─────────
│ 1973-01-31 │    missing │     missing │      missing │   0.00235344 │     mi ⋯
│ 1973-02-28 │    missing │     missing │      missing │   0.00704374 │     mi ⋯
│ 1973-03-31 │    missing │     missing │      missing │   0.00932044 │     mi ⋯
│ 1973-04-30 │    missing │     missing │      missing │   0.00692989 │     mi ⋯
│ 1973-05-31 │    missing │     missing │      missing │   0.00688219 │     mi ⋯
│ 1973-06-30 │    missing │     missing │      missing │   0.00683515 │     mi ⋯
│ 1973-07-31 │    missing │     missing │      missing │   0.00225755 │     mi ⋯
│ 1973-08-31 │    missing │     missing │      missing │    0.0180626 │     mi ⋯
│     ⋮      │     ⋮      │      ⋮      │

In [40]:
colnames(data)

8-element Vector{Symbol}:
 :ACWI_Net
 :USDPLN
 :ACWI_Net_PLN
 :CPI_USA
 :CPI_PLN
 :TBSP
 :WIBOR3m
 :TenYPL

In [237]:
function returns_summarystats(data::TimeArray,t)
    names = colnames(data)
    returns = transpose(values(data))
    n_assets = size(returns)[1]

    stats = [ Dict(
        :mean => mean(returns[i,:]) * t, 
        :std => std(returns[i,:]) * t^0.5,
        :median => median(returns[i,:] * t),
        :skewness => skewness(returns[i,:]),
        :kurtosis => kurtosis(returns[i,:]),
        :autocor => autocor(returns[i,:],[1])[1],
        :p25th => percentile(returns[i,:],25) * t,
        :p75th => percentile(returns[i,:],75) * t,
        :min => minimum(returns[i,:]) * t,
        :max => maximum(returns[i,:]) * t,
        ) for i in 1:n_assets ]

    short_stats = pretty_table(stats, backend = Val(:html), row_labels = names)
    return short_stats
end



returns_summarystats (generic function with 1 method)

In [238]:
short = Float64.(from(data[:TBSP, :WIBOR3m, :TenYPL],Date(2007,01,01)))

207×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 2007-01-31 to 2024-03-31
┌────────────┬─────────────┬────────────┬────────────┐
│            │ TBSP        │ WIBOR3m    │ TenYPL     │
├────────────┼─────────────┼────────────┼────────────┤
│ 2007-01-31 │     0.00626 │ 0.00349167 │     0.0043 │
│ 2007-02-28 │ 0.000149067 │ 0.00351667 │ 0.00435333 │
│ 2007-03-31 │  0.00510726 │ 0.00353333 │ 0.00434083 │
│ 2007-04-30 │ 0.000692007 │ 0.00366667 │ 0.00443917 │
│ 2007-05-31 │  0.00362559 │     0.0037 │ 0.00443917 │
│ 2007-06-30 │ -0.00847508 │   0.003925 │ 0.00470083 │
│ 2007-07-31 │  0.00471553 │ 0.00400833 │ 0.00466583 │
│ 2007-08-31 │ -0.00094856 │ 0.00419167 │ 0.00482417 │
│     ⋮      │      ⋮      │     ⋮      │     ⋮      │
│ 2023-09-30 │  0.00622425 │ 0.00480833 │ 0.00491583 │
│ 2023-10-31 │   0.0057372 │ 0.00471667 │ 0.00471167 │
│ 2023-11-30 │  0.00315821 │ 0.00485833 │   0.004585 │
│ 2023-12-31 │   0.0171423 │     0.0049 │  0.0043475 │
│ 2024-01-31 │   0.0045132 │ 0.00488333 │   

In [239]:
returns_summarystats(short,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
TBSP,0.0428527,-0.502584,0.827558,0.0403455,0.342799,-0.0282004,0.0553435,5.80245,0.104502,0.0448318
WIBOR3m,0.00580118,0.0021,0.0751,0.0337908,0.271898,0.0172,0.988687,-1.03315,0.04915,0.0274
TenYPL,0.00476751,0.01149,0.08337,0.042673,-0.144031,0.02954,0.977263,-1.17253,0.057915,0.04218


In [199]:
varls = to(Float64.(from(data[:WIBOR3m, :TenYPL],Date(2003,01,01))),Date(2006,12,31))
returns_summarystats(varls,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
WIBOR3m,0.0026271,0.0413,0.07,0.0532854,0.168329,0.04475,0.937251,-1.21056,0.05985,0.0542
TenYPL,0.00246801,0.04622,0.07497,0.0581775,0.642365,0.05172,0.912227,-0.865177,0.0661475,0.055605


In [200]:
varls

48×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬────────────┬────────────┐
│            │ WIBOR3m    │ TenYPL     │
├────────────┼────────────┼────────────┤
│ 2003-01-31 │ 0.00539167 │ 0.00477833 │
│ 2003-02-28 │   0.005225 │  0.0046275 │
│ 2003-03-31 │ 0.00498333 │   0.004565 │
│ 2003-04-30 │ 0.00474167 │ 0.00441583 │
│ 2003-05-31 │     0.0045 │ 0.00421583 │
│ 2003-06-30 │ 0.00443333 │  0.0042525 │
│ 2003-07-31 │ 0.00436667 │ 0.00472417 │
│ 2003-08-31 │ 0.00428333 │ 0.00471167 │
│     ⋮      │     ⋮      │     ⋮      │
│ 2006-06-30 │ 0.00350833 │  0.0047375 │
│ 2006-07-31 │ 0.00348333 │  0.0046675 │
│ 2006-08-31 │ 0.00349167 │ 0.00466167 │
│ 2006-09-30 │ 0.00351667 │ 0.00459167 │
│ 2006-10-31 │ 0.00350833 │  0.0044175 │
│ 2006-11-30 │     0.0035 │ 0.00431083 │
│ 2006-12-31 │     0.0035 │  0.0043525 │
└────────────┴────────────┴────────────┘
                         33 rows omitted

In [145]:
ols = lm(@formula(TBSP ~ WIBOR3m + TenYPL), short[:TBSP, :WIBOR3m, :TenYPL])

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

TBSP ~ 1 + WIBOR3m + TenYPL

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   0.00787004  0.00266261   2.96    0.0035   0.00262028   0.0131198
WIBOR3m       4.73141     1.18903      3.98    <1e-04   2.38705      7.07577
TenYPL       -5.01425     1.44683     -3.47    0.0006  -7.86691     -2.16159
──────────────────────────────────────────────────────────────────────────────

In [152]:
println(round(r2(ols),digits=4))


0.0726


In [163]:
res = residuals(ols)

207-element Vector{Float64}:
  0.003430716591018009
 -0.0025310752522496593
  0.002285585345042713
 -0.0022674570120775547
  0.0005084084603820815
 -0.011344760884167953
  0.0012760621939868766
 -0.004461530558817827
  0.0027820739511106307
  0.0011500434712708452
  ⋮
  0.005338244277345139
 -0.009133260284113345
  0.00025321597553191674
 -0.0008238605338106754
 -0.004708281211956773
  0.007887752722751981
 -0.004624845680767025
 -0.011182006298309195
 -0.00807717870245278

In [164]:
deviance(ols)

0.029236942219919678

In [169]:
(sum(res.^2)/(207-3))

0.00014331834421529255

In [196]:
predicted = predict(ols,varls)
display(mean(predicted)*12)
display(std(predicted)*12^0.5)

0.05483928570743096

0.010535360864781951

In [210]:
random_res = rand(res,48)

48-element Vector{Float64}:
  0.007541879829052814
  0.00838751717896534
  0.007617190368768793
  0.0012763440233279095
 -0.006146694887943102
  0.00766903386254357
 -0.003733571619440039
  0.0054204698345044
  0.006914983209426668
  0.008704404950540815
  ⋮
  0.0029304362296530898
  0.0027680026157853865
 -0.0006618217668654981
  0.0009491984375571451
  0.00043626061176266565
 -0.00953052919815306
 -0.004470823475361239
 -0.01030063143396127
  0.0001707473214432268

In [227]:
backfilled = predicted + random_res
display(mean(backfilled)*12)
display(std(backfilled)*12^0.5)

0.05191826057329639

0.04853954192730064

In [229]:
backfilled = TimeArray(timestamp(varls),backfilled,[:TBSP])

48×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬─────────────┐
│            │ TBSP        │
├────────────┼─────────────┤
│ 2003-01-31 │   0.0169624 │
│ 2003-02-28 │   0.0177758 │
│ 2003-03-31 │   0.0161754 │
│ 2003-04-30 │  0.00943908 │
│ 2003-05-31 │  0.00187546 │
│ 2003-06-30 │   0.0151919 │
│ 2003-07-31 │  0.00110882 │
│ 2003-08-31 │  0.00993126 │
│     ⋮      │      ⋮      │
│ 2006-06-30 │  5.25843e-5 │
│ 2006-07-31 │  0.00189632 │
│ 2006-08-31 │  0.00145206 │
│ 2006-09-30 │ -0.00804545 │
│ 2006-10-31 │ -0.00215186 │
│ 2006-11-30 │ -0.00748624 │
│ 2006-12-31 │  0.00277621 │
└────────────┴─────────────┘
             33 rows omitted

In [243]:
ls = merge(backfilled, varls)

48×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬─────────────┬────────────┬────────────┐
│            │ TBSP        │ WIBOR3m    │ TenYPL     │
├────────────┼─────────────┼────────────┼────────────┤
│ 2003-01-31 │   0.0169624 │ 0.00539167 │ 0.00477833 │
│ 2003-02-28 │   0.0177758 │   0.005225 │  0.0046275 │
│ 2003-03-31 │   0.0161754 │ 0.00498333 │   0.004565 │
│ 2003-04-30 │  0.00943908 │ 0.00474167 │ 0.00441583 │
│ 2003-05-31 │  0.00187546 │     0.0045 │ 0.00421583 │
│ 2003-06-30 │   0.0151919 │ 0.00443333 │  0.0042525 │
│ 2003-07-31 │  0.00110882 │ 0.00436667 │ 0.00472417 │
│ 2003-08-31 │  0.00993126 │ 0.00428333 │ 0.00471167 │
│     ⋮      │      ⋮      │     ⋮      │     ⋮      │
│ 2006-06-30 │  5.25843e-5 │ 0.00350833 │  0.0047375 │
│ 2006-07-31 │  0.00189632 │ 0.00348333 │  0.0046675 │
│ 2006-08-31 │  0.00145206 │ 0.00349167 │ 0.00466167 │
│ 2006-09-30 │ -0.00804545 │ 0.00351667 │ 0.00459167 │
│ 2006-10-31 │ -0.00215186 │ 0.00350833 │  0.

In [245]:
CSV.write("tbsp_backfilled.csv",ls)

"tbsp_backfilled.csv"

In [244]:
long = vcat(short,ls)

TypeError: TypeError: non-boolean (Missing) used in boolean context